# El método `.shift()` de pandas
En **pandas**, el método **`.shift()`** es una herramienta fundamental para **desplazar (rezagar o adelantar) una serie de tiempo**, y por eso es clave cuando construyes **atributos meteorológicos con rezago** para modelos estadísticos o de *machine learning* (ARIMA/SARIMAX, regresión, Random Forest, SVM, etc.).



Dado tu trabajo con **series meteorológicas y epidemiológicas**, este método aparece en casi todas las etapas de *feature engineering* temporal.

---

## 1. ¿Qué hace `.shift()`?



```python
Series.shift(periods=1, freq=None)
DataFrame.shift(periods=1, freq=None)
```



* **`periods`**: número de pasos que se desplaza la serie

  * positivo → **rezago (lag)**
  * negativo → **adelanto (lead)**
* **`freq`**: desplaza el índice temporal (no los datos), se usa menos en ML

📌 **Idea clave**:

> `.shift()` **no mezcla información del futuro**, por lo que evita *data leakage*.

---

## 2. Ejemplo básico con una serie meteorológica

Supongamos una serie diaria de **temperatura media**:



```python
import pandas as pd

fechas = pd.date_range("2024-01-01", periods=5, freq="D")
temp = pd.Series([28, 29, 30, 31, 32], index=fechas, name="temp_media")

temp
```



### Rezago de 1 día



```python
temp.shift(1)
```



Resultado conceptual:

| Fecha      | temp_media | temp_media_lag1 |
| ---------- | ---------- | --------------- |
| 2024-01-01 | 28         | NaN             |
| 2024-01-02 | 29         | 28              |
| 2024-01-03 | 30         | 29              |
| 2024-01-04 | 31         | 30              |
| 2024-01-05 | 32         | 31              |

📌 El **NaN inicial** aparece porque no existe información previa.

---



## 3. `.shift()` en DataFrames meteorológicos

Con varias variables:



```python
df = pd.DataFrame({
    "temp": [28, 29, 30, 31, 32],
    "humedad": [70, 72, 75, 78, 80],
    "precipitacion": [0, 5, 0, 10, 2]
}, index=fechas)
```



## Crear rezagos de un día



```python
df_lag1 = df.shift(1)
```



O agregarlos al mismo DataFrame:



```python
df["temp_lag1"] = df["temp"].shift(1)
df["humedad_lag1"] = df["humedad"].shift(1)
df["precip_lag1"] = df["precipitacion"].shift(1)
```



---

## 4. Creación sistemática de atributos con rezago

Muy útil en proyectos como **alertas tempranas epidemiológicas**:



```python
for lag in [1, 3, 7, 14]:
    df[f"temp_lag{lag}"] = df["temp"].shift(lag)
    df[f"humedad_lag{lag}"] = df["humedad"].shift(lag)
```



📌 Interpretación:

* `lag 1` → efecto inmediato
* `lag 7` → efecto semanal
* `lag 14` → incubación o efectos retardados

---



## 5. Uso didáctico: interpretación causal

En meteorología–salud:

> “El número de casos hoy **no depende del clima de hoy**, sino del clima de **hace varios días**”.

Formalmente:

$$
\text{Casos}*t = f(\text{Temp}*{t-7}, \text{Hum}*{t-10}, \text{Prec}*{t-14})
$$

En código:



```python
df["temp_lag7"] = df["temp"].shift(7)
df["humedad_lag10"] = df["humedad"].shift(10)
df["precip_lag14"] = df["precipitacion"].shift(14)
```



---

## 6. Diferencia entre `.shift()` y `.diff()`

| Método     | Qué hace                   | Uso típico                 |
| ---------- | -------------------------- | -------------------------- |
| `.shift()` | Desplaza la serie          | Rezagos, causalidad        |
| `.diff()`  | Resta valores consecutivos | Tendencia, estacionariedad |

Ejemplo combinado:



```python
df["temp_cambio"] = df["temp"].diff()
df["temp_lag1"] = df["temp"].shift(1)
```



---

## 7. `.shift()` y modelos de ML



### Matriz de entrenamiento



```python
X = df[[
    "temp_lag1", "temp_lag7",
    "humedad_lag1", "humedad_lag7"
]]

y = df["casos"]
```



⚠️ **Siempre eliminar NaN antes de entrenar**:



```python
Xy = pd.concat([X, y], axis=1).dropna()
```



---

## 8. `.shift(freq=...)` (menos usado)



```python
df.shift(freq="D")
```



* Desplaza el **índice temporal**
* No mueve los valores
* Útil para alineaciones de calendario, no para ML



---

## 9. Buenas prácticas en proyectos reales

✔ Crear rezagos coherentes con el fenómeno
✔ Documentar el significado físico de cada lag
✔ Evitar usar valores futuros (`shift(-1)`)
✔ Probar múltiples ventanas (1, 7, 14, 21, 28 días)
✔ Combinar con promedios móviles (`rolling()`)



Ejemplo potente:

```python


df["temp_lag7"] = df["temp"].shift(7)
df["temp_media_7d"] = df["temp"].shift(1).rolling(7).mean()


```

---

## 10. Mensaje clave para tus estudiantes

> **`.shift()` transforma una serie de tiempo en un conjunto de variables explicativas que respetan el tiempo y permiten modelar relaciones causales.**

Si quieres, en el siguiente mensaje puedo:

* Proponerte **ejercicios didácticos** con datos meteorológicos
* Mostrar un **caso completo SARIMAX**
* Conectar `.shift()` con **Random Forest o SVM** para predicción de casos epidemiológicos
